In [1]:
import pandas as pd
from fbprophet import Prophet
import http.client, urllib.request, urllib.parse, urllib.error, base64
import re

PROPHET METHOD

In [2]:
def sasi_Prophet(filepath):
    
    df = pd.read_csv(filepath)
    df.columns=['ds', 'y']
    m = Prophet()
    m.fit(df)
    future_dates = m.make_future_dataframe(periods=30)
    future_dates.tail()
    forecast = m.predict(future_dates)
    result=(forecast[['ds', 'yhat']].tail(30))
    result.columns=['date','Temp']
    result=result.set_index(['date'])
    result.to_csv('kac/'+filepath, sep=',')    

In [3]:
sasi_Prophet('Phoenix.csv')

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
F:\Anaconda\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [4]:
def getFlightDetails(to,year,month,day):
    
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': '00c12c7f625f4eeaa65bac14b53489d7',
    }
    body=""

    params = urllib.parse.urlencode({
        # Request parameters
        'Connection': 'DIRECT'
    })

    try:
        conn = http.client.HTTPSConnection('flightlookup.azure-api.net')
        conn.request("GET", "/v1/xml/TimeTable/JFK/"+to+"/"+year+month+day+"/?%s" % params,body , headers)
        response = conn.getresponse()
        data = response.read()
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    #get flight timings
    dataString=str(data)
    flightDurations = re.findall('TotalFlightTime="PT(.+?)"', dataString)
    
    #get fliht names
    flights = re.findall('CompanyShortName="(.+?)" />', dataString)
    
    #store flight details into a pandas dataframe
    flightData = pd.DataFrame(
        {
         'Duration': flightDurations[0:3],
         'Flights': flights[0:3],
        })
    return(flightData)


In [5]:
def main(date,temp):
    first = pd.read_csv('kac/Phoenix.csv',index_col=0)
    dictfirst=first.to_dict()
    second = pd.read_csv('kac/San Diego.csv',index_col=0)
    dictsecond=second.to_dict()
    third = pd.read_csv('kac/Seattle.csv',index_col=0)
    dictthird=third.to_dict()
    
    firstTemp = dictfirst['Temp'][date]
    secondTemp = dictsecond['Temp'][date]
    thirdTemp =  dictthird['Temp'][date]
    
    tempdif1 = temp-firstTemp
    tempdif2 = temp-secondTemp
    tempdif3 = temp-thirdTemp
    
    Array = {"PHX":tempdif2,"SAN":tempdif1,"SEA":tempdif3}
    sortedDictTempDiff=sorted(Array.values())
    
    for key,value in Array.items():
        if (value==sortedDictTempDiff[0]):
            firstcity=key
        
        if (value==sortedDictTempDiff[1]):
            secondcity=key
        
               
  
    city = {"PHX":'Phoenix',"SAN":'SanDieago',"SEA":'Seattle'}
    
    year=date[0:4]
    month=date[5:7]
    day=date[8:]
    
    
    print("---------------------First Preference-------------------")
    print ("\nTemperature is :",temp+Array[firstcity])
    print ("City is",city[firstcity])
    print("\n",getFlightDetails(firstcity,'2019',month,day))
    
    print("\n\n---------------------Second Preference-------------------")
    print ("\nTemperature is :",temp+Array[secondcity])
    print ("City is",city[secondcity])
    print("\n",getFlightDetails(secondcity,'2019',month,day))
       

In [6]:
main('2017-12-24',278)

---------------------First Preference-------------------

Temperature is : 270.38776030967944
City is Phoenix

   Duration            Flights
0    5H19M  American Airlines
1    5H47M  American Airlines
2    6H00M    Delta Air Lines


---------------------Second Preference-------------------

Temperature is : 270.59999421227656
City is SanDieago

   Duration          Flights
0    6H38M  Delta Air Lines
1    6H27M  JetBlue Airways
2    6H23M  JetBlue Airways
